In [1]:
import networkx as nx
import numpy as np
from network import ConstantBirthNode, DynamicBirthNode, Network
from reactions import NetworkReactions, CmeParameters
from simulate import simulate_gillespie

ImportError: Python version mismatch: module was compiled for Python 3.9, but the interpreter version is incompatible: 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0].

In [3]:
node0 = ConstantBirthNode(60, 0.2, 0)
node1 = DynamicBirthNode(60, 0.2, 0.2, 60, 0.1, 0.5)
node2 = DynamicBirthNode(60, 0.1, 0.1, 60, 0.1, 0.5)

network = Network()
network.add_node(node0)
network.add_node(node1)
network.add_node(node2)
network.allocate_ids()

network.add_transport(node0, node1, rate = 0.05)
network.add_transport(node1, node0, rate = 0.05)
network.add_transport(node1, node2, rate = 0.03)
network.add_transport(node2, node1, rate = 0.03)


for node in network.nodes: print(node)
for s, d, data in network.edges(data=True):
    print(s, d, data['rate'])

reactions = NetworkReactions(network)

print("\nReactions:")
for reaction in reactions.reactions:
    print(reaction)

cme_param = CmeParameters(reactions)
print(cme_param)

ConstantBirthNode(id:0)
DynamicBirthNode(id:1)
DynamicBirthNode(id:2)
ConstantBirthNode(id:0) DynamicBirthNode(id:1) 0.05
DynamicBirthNode(id:1) ConstantBirthNode(id:0) 0.05
DynamicBirthNode(id:1) DynamicBirthNode(id:2) 0.03
DynamicBirthNode(id:2) DynamicBirthNode(id:1) 0.03

Reactions:
DynamicBirthReaction(variable:n1_t0, state_i:2, rate:dynamic)
DynamicBirthReaction(variable:n1_t1, state_i:3, rate:dynamic)
DynamicBirthReaction(variable:n2_t0, state_i:4, rate:dynamic)
DynamicBirthReaction(variable:n2_t1, state_i:5, rate:dynamic)
DeathReaction(variable:n0_t0, state_i:0, rate:0.2)
DeathReaction(variable:n0_t1, state_i:1, rate:0.2)
DeathReaction(variable:n1_t0, state_i:2, rate:0.2)
DeathReaction(variable:n1_t1, state_i:3, rate:0.2)
DeathReaction(variable:n2_t0, state_i:4, rate:0.1)
DeathReaction(variable:n2_t1, state_i:5, rate:0.1)
TransportReaction(variable:n0_t0, state_i:0, sourceid: 0, destid: 1, rate:0.05)
TransportReaction(variable:n0_t1, state_i:1, sourceid: 0, destid: 1, rate:0.05

In [ ]:

simulate = 